# Chap6.2及び6.4




In [1]:
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# 追記
import json
import datetime
import math
plt.style.use('dark_background')

# debug
#%pdb on

import pixiedust #%pixie_debugger
import os
os.environ["CUDA_VISIBLE_DEVICES"]="5"

# tfがエラーはかないため
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
# physical_devices = tensorflow.keras.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)


Pixiedust database opened successfully


In [2]:
import tensorflow as tf
from tensorflow.keras import Model, layers
# from tf.keras import Model, layers
import tensorflow.keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
# import matplotlib.pyplot as plt



In [3]:
max_features = 10000
max_len = 500
BATCH_SIZE = 128
EPOCH = 10

In [4]:
(input_train, y_train), ( input_test, y_test) = imdb.load_data(num_words=max_features)

x_train = sequence.pad_sequences(input_train, maxlen=max_len)
x_test = sequence.pad_sequences(input_test, maxlen=max_len)

In [5]:
x_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

０のPadding（？）は後ではなく前に足されるというのに注意

In [6]:
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_reveiw = ' '.join([reverse_word_index.get(i-3, '?') for  i in x_train[0]])
decoded_reveiw

"? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the fil

In [7]:
x_train.shape

(25000, 500)

## Chap6.2
以下の3つについて説明\
SimpleRNN
LSTM
GRU　

### 6.2.1 SimpleRNNの場合
https://www.tensorflow.org/api_docs/python/tf/keras/layers/SimpleRNN

※activationはtahnが標準設定

In [8]:
class SimpleRNN_model():
  def __init__(self, BATCH_SIZE, EPOCH):
    self.epoch = EPOCH
    self.batch_size = BATCH_SIZE
    self.model = self._build_model()
    self.model.summary()
    self.model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    
  def _build_model(self):
    input = tf.keras.Input(shape=(500,))
    x = tf.keras.layers.Embedding(max_features,32)(input)
    x = tf.keras.layers.SimpleRNN(32)(x)
    y_pred = tf.keras.layers.Dense(1,activation='sigmoid')(x)

    return Model(input, y_pred)
  
  def train(self, x, y):
    history = self.model.fit(x , y ,epochs=self.epoch, batch_size=self.batch_size, validation_split=0.2)
    return history

model = SimpleRNN_model(BATCH_SIZE,EPOCH)

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 500, 32)           320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 322,113
Trainable params: 322,113
Non-trainable params: 0
_________________________________________________________________


In [9]:
history1 = model.train(x_train,y_train)

Epoch 1/10
157/157 [==============================] - 43s 273ms/step - loss: 0.6354 - acc: 0.6253 - val_loss: 0.5076 - val_acc: 0.7680
Epoch 2/10
157/157 [==============================] - 42s 265ms/step - loss: 0.3910 - acc: 0.8339 - val_loss: 0.4498 - val_acc: 0.7894
Epoch 3/10
157/157 [==============================] - 43s 273ms/step - loss: 0.2887 - acc: 0.8820 - val_loss: 0.4160 - val_acc: 0.8206
Epoch 4/10
157/157 [==============================] - 43s 272ms/step - loss: 0.2194 - acc: 0.9168 - val_loss: 0.3763 - val_acc: 0.8456
Epoch 5/10
157/157 [==============================] - 42s 266ms/step - loss: 0.1534 - acc: 0.9454 - val_loss: 0.4403 - val_acc: 0.8332
Epoch 6/10
157/157 [==============================] - 42s 268ms/step - loss: 0.1017 - acc: 0.9661 - val_loss: 0.4833 - val_acc: 0.8100
Epoch 7/10
157/157 [==============================] - 42s 266ms/step - loss: 0.0696 - acc: 0.9768 - val_loss: 0.6710 - val_acc: 0.7450
Epoch 8/10
157/157 [==============================] - 4

### 6.2.2 LSTMの場合
https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM

※Activation層はtahnが標準設定されていることに注意
※recurrent_activationは、'sigmoid'で標準設定

In [10]:
class LSTM_model():
  def __init__(self, BATCH_SIZE, EPOCH):
    self.epoch = EPOCH
    self.batch_size = BATCH_SIZE
    self.model = self._build_model()
    self.model.summary()
    self.model.compile(optimizer='rmsprop', loss='binary_crossentropy',metrics=['acc'])
  
  def _build_model(self):
    input = tf.keras.Input(shape=(500,))
    x = layers.Embedding(max_features,32)(input)
    x = layers.LSTM(32)(x)
    y_pred = layers.Dense(1,activation='sigmoid')(x)
    return Model(input, y_pred)
  
  def train(self, x, y):
    history = self.model.fit(x , y ,epochs=self.epoch, batch_size=self.batch_size, validation_split=0.2)
    
    return history

model2 = LSTM_model(BATCH_SIZE,EPOCH)

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 328,353
Trainable params: 328,353
Non-trainable params: 0
_________________________________________________________________


In [11]:
history2 = model2.train(x_train,y_train)

Epoch 1/10
157/157 [==============================] - 7s 46ms/step - loss: 0.5255 - acc: 0.7487 - val_loss: 0.3607 - val_acc: 0.8668
Epoch 2/10
157/157 [==============================] - 7s 43ms/step - loss: 0.3092 - acc: 0.8792 - val_loss: 0.3102 - val_acc: 0.8696
Epoch 3/10
157/157 [==============================] - 7s 43ms/step - loss: 0.2408 - acc: 0.9103 - val_loss: 0.2912 - val_acc: 0.8764
Epoch 4/10
157/157 [==============================] - 7s 44ms/step - loss: 0.2028 - acc: 0.9237 - val_loss: 0.2823 - val_acc: 0.8862
Epoch 5/10
157/157 [==============================] - 7s 44ms/step - loss: 0.1779 - acc: 0.9341 - val_loss: 0.3335 - val_acc: 0.8846
Epoch 6/10
157/157 [==============================] - 7s 44ms/step - loss: 0.1600 - acc: 0.9431 - val_loss: 0.3001 - val_acc: 0.8784
Epoch 7/10
157/157 [==============================] - 7s 44ms/step - loss: 0.1417 - acc: 0.9485 - val_loss: 0.6407 - val_acc: 0.8054
Epoch 8/10
157/157 [==============================] - 7s 43ms/step - 

###6.2.3 GRUの場合
https://www.tensorflow.org/api_docs/python/tf/keras/layers/GRU

※Activation層はtahnが標準設定されていることに注意\
※recurrent_activationは、'sigmoid'で標準設定

In [12]:
class GRU_model():
  def __init__(self, BATCH_SIZE, EPOCH):
    self.epoch = EPOCH
    self.batch_size = BATCH_SIZE
    self.model = self._build_model()
    self.model.summary()
    self.model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    
  def _build_model(self):
    input = tf.keras.Input(shape=(500,))
    x = layers.Embedding(max_features,32)(input)
    x = layers.GRU(32)(x)
    y_pred = layers.Dense(1,activation='sigmoid')(x)

    return Model(input, y_pred)
  
  def train(self, x, y):
    history = self.model.fit(x , y ,epochs=self.epoch, batch_size=self.batch_size, validation_split=0.2)
    return history

model3 = GRU_model(BATCH_SIZE,EPOCH)

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
gru (GRU)                    (None, 32)                6336      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 326,369
Trainable params: 326,369
Non-trainable params: 0
_________________________________________________________________


In [13]:
history3 = model3.train(x_train,y_train)

Epoch 1/10
157/157 [==============================] - 7s 46ms/step - loss: 0.5622 - acc: 0.7053 - val_loss: 0.3989 - val_acc: 0.8224
Epoch 2/10
157/157 [==============================] - 7s 43ms/step - loss: 0.3329 - acc: 0.8625 - val_loss: 0.3281 - val_acc: 0.8682
Epoch 3/10
157/157 [==============================] - 7s 43ms/step - loss: 0.2607 - acc: 0.8964 - val_loss: 0.3438 - val_acc: 0.8570
Epoch 4/10
157/157 [==============================] - 7s 43ms/step - loss: 0.2190 - acc: 0.9168 - val_loss: 0.3759 - val_acc: 0.8556
Epoch 5/10
157/157 [==============================] - 7s 43ms/step - loss: 0.1911 - acc: 0.9280 - val_loss: 0.6309 - val_acc: 0.7890
Epoch 6/10
157/157 [==============================] - 7s 43ms/step - loss: 0.1708 - acc: 0.9374 - val_loss: 0.3223 - val_acc: 0.8860
Epoch 7/10
157/157 [==============================] - 7s 44ms/step - loss: 0.1584 - acc: 0.9416 - val_loss: 0.3212 - val_acc: 0.8824
Epoch 8/10
157/157 [==============================] - 7s 43ms/step - 

###6.2.4 Bidirectional_model
Chap6.3で説明\
https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional
引数がLayerになっていることに留意


In [14]:
class Bidirectional_model():
  def __init__(self, BATCH_SIZE, EPOCH):
    self.epoch = EPOCH
    self.batch_size = BATCH_SIZE
    self.model = self._build_model()
    self.model.summary()
    self.model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    
  def _build_model(self):
    input = tf.keras.Input(shape=(500,))
    x = layers.Embedding(max_features,32)(input)
    #引数にはLayerをいれる
    x = layers.Bidirectional(layers.LSTM(32))(x)
    y_pred = layers.Dense(1,activation='sigmoid')(x)

    return Model(input, y_pred)
  
  def train(self, x, y):
    history = self.model.fit(x , y ,epochs=self.epoch, batch_size=self.batch_size, validation_split=0.2)
    return history

model4 = Bidirectional_model(BATCH_SIZE,EPOCH)

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 64)                16640     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 336,705
Trainable params: 336,705
Non-trainable params: 0
_________________________________________________________________


In [15]:
history4 = model4.train(x_train,y_train)

Epoch 1/10
157/157 [==============================] - 12s 77ms/step - loss: 0.5671 - acc: 0.7246 - val_loss: 0.4595 - val_acc: 0.7922
Epoch 2/10
157/157 [==============================] - 11s 73ms/step - loss: 0.3207 - acc: 0.8747 - val_loss: 0.3564 - val_acc: 0.8412
Epoch 3/10
157/157 [==============================] - 12s 74ms/step - loss: 0.2450 - acc: 0.9051 - val_loss: 0.3353 - val_acc: 0.8602
Epoch 4/10
157/157 [==============================] - 12s 74ms/step - loss: 0.2061 - acc: 0.9247 - val_loss: 0.3433 - val_acc: 0.8810
Epoch 5/10
157/157 [==============================] - 12s 75ms/step - loss: 0.1807 - acc: 0.9362 - val_loss: 0.3626 - val_acc: 0.8696
Epoch 6/10
157/157 [==============================] - 12s 76ms/step - loss: 0.1560 - acc: 0.9427 - val_loss: 0.3296 - val_acc: 0.8838
Epoch 7/10
157/157 [==============================] - 12s 76ms/step - loss: 0.1473 - acc: 0.9496 - val_loss: 0.3394 - val_acc: 0.8836
Epoch 8/10
157/157 [==============================] - 12s 75ms

## Chap6.4

### 6.4.1 Conv1Dのみの単純なModel

In [16]:
class SimpleConv1d_model():
  def __init__(self, BATCH_SIZE, EPOCH):
    self.epoch = EPOCH
    self.batch_size = BATCH_SIZE
    self.model = self._build_model()
    self.model.summary()
    self.model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    
  def _build_model(self):
    input = tf.keras.Input(shape=(500,))
    x = layers.Embedding(max_features,32)(input)
    x = layers.Conv1D(32,7,activation='relu',padding='same')(x)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(32,7,activation='relu',padding='same')(x)
    x = layers.GlobalMaxPooling1D()(x)
    y_pred = layers.Dense(1,activation='sigmoid')(x)

    return Model(input, y_pred)
  
  def train(self, x, y):
    history = self.model.fit(x , y ,epochs=self.epoch, batch_size=self.batch_size, validation_split=0.2)
    return history

model5 = SimpleConv1d_model(BATCH_SIZE,EPOCH)

Model: "functional_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
conv1d (Conv1D)              (None, 500, 32)           7200      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 100, 32)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 32)           7200      
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                

In [17]:
history5 = model5.train(x_train,y_train)

Epoch 1/10
157/157 [==============================] - 2s 16ms/step - loss: 0.5346 - acc: 0.7253 - val_loss: 0.3527 - val_acc: 0.8466
Epoch 2/10
157/157 [==============================] - 2s 14ms/step - loss: 0.2782 - acc: 0.8861 - val_loss: 0.3144 - val_acc: 0.8658
Epoch 3/10
157/157 [==============================] - 2s 14ms/step - loss: 0.2092 - acc: 0.9188 - val_loss: 0.3179 - val_acc: 0.8714
Epoch 4/10
157/157 [==============================] - 2s 14ms/step - loss: 0.1672 - acc: 0.9370 - val_loss: 0.3346 - val_acc: 0.8692
Epoch 5/10
157/157 [==============================] - 2s 14ms/step - loss: 0.1373 - acc: 0.9507 - val_loss: 0.3109 - val_acc: 0.8836
Epoch 6/10
157/157 [==============================] - 2s 14ms/step - loss: 0.1126 - acc: 0.9617 - val_loss: 0.3686 - val_acc: 0.8738
Epoch 7/10
157/157 [==============================] - 2s 14ms/step - loss: 0.0891 - acc: 0.9714 - val_loss: 0.3996 - val_acc: 0.8672
Epoch 8/10
157/157 [==============================] - 2s 14ms/step - 

### 6.4.2 CNNとRNNを組み合わせた長いModel

In [18]:
class Conv1d_model2():
  def __init__(self, BATCH_SIZE, EPOCH):
    self.epoch = EPOCH
    self.batch_size = BATCH_SIZE
    self.model = self._build_model()
    self.model.summary()
    self.model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['acc'])
    
  def _build_model(self):
    input = tf.keras.Input(shape=(500,))
    x = layers.Embedding(max_features,32)(input)
    x = layers.Conv1D(32,5, activation='relu', padding='same')(x)
    x = layers.MaxPooling1D(5)(x)
    x = layers.Conv1D(32,5,activation='relu',padding='same')(x)
    x = layers.GRU(32,dropout=0.1,recurrent_dropout=0.5)(x)
    y_pred = layers.Dense(1,activation='sigmoid')(x)

    return Model(input, y_pred)
  
  def train(self, x, y):
    history = self.model.fit(x , y ,epochs=self.epoch, batch_size=self.batch_size, validation_split=0.2)
    return history

model6 = Conv1d_model2(BATCH_SIZE,EPOCH)

Model: "functional_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 500)]             0         
_________________________________________________________________
embedding_5 (Embedding)      (None, 500, 32)           320000    
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 32)           5152      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 100, 32)           0         
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 100, 32)           5152      
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                6336      
_________________________________________________________________
dense_5 (Dense)              (None, 1)               

In [ ]:
history6 = model6.train(x_train,y_train)

Epoch 1/10
 80/157 [==============>...............] - ETA: 23s - loss: 0.6248 - acc: 0.6321

In [ ]:
def make_graph(history_list):
  name_list = ['RNN', 'LSTM', 'GRU', 'Conv1', 'Conv+RNN']
  fig = plt.figure(figsize=(16,8))

  plt.subplots_adjust(wspace=0.2, hspace=0.4)

  for i in range(len(name_list)):
    ax = fig.add_subplot(2,3,i+1)
    loss = history_list[i].history['loss']
    val_loss = history_list[i].history['val_loss']
    epochs = range(1,len(loss)+1)
    ax.plot(epochs, loss, linewidth=1,linestyle='--',label='training_loss')
    ax.plot(epochs, val_loss, linewidth=1, label='validation_loss')
    ax.grid(True)

    ax.set_title(name_list[i],fontsize=20)
    ax.set_xlabel('Epochs',fontsize=15)
    ax.set_ylabel('Loss',fontsize=15)
    ax.legend(handlelength=3,loc='upper right')
  
  plt.show()

In [ ]:
history_list = [history1,history2,history3,history5,history6]
make_graph(history_list)